# 目標：從空拍圖裡找出鯨魚，將鯨魚和大海分割。

![img_whales](https://kaggle2.blob.core.windows.net/competitions/kaggle/4521/media/ChristinKhan_RightWhaleMomCalf_640.png)

我們將建立一個完全不使用Dense層的Fully Convolutional Network (FCN)。

數據來源：

https://www.kaggle.com/c/noaa-right-whale-recognition

## 本筆記內容如下

* [I. 宣告模型架構及訓練方式](#31)
* [II. 訓練模型三步驟：1. 將圖像準備好成為適合訓練的格式 2. 訓練模型 3. 畫出訓練過程](#32)
* [III. 檢驗模型好壞](#33)
* [IV. 拿訓練好的網路來做預測](#34)

---

In [ ]:
# # =====================================================================
# # 由於課堂上可能有多人共用同一顆GPU，以下限定使用者只能使用計算卡上面一半的記憶體。
# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.5 # 使用一半記憶體
# set_session(tf.Session(config=config))
# # =====================================================================

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import os,time,json
from sklearn.metrics import classification_report

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import model_from_json
from keras.optimizers import SGD
from keras.layers import Conv2D
from whales_model import SoftmaxMap, my_ConvNet

---

### <a id='31'>I. 宣告模型架構及訓練方式</a>

In [ ]:
# 建立模型
input_shape=(227,227,3)
model=my_ConvNet(input_shape) # 一個簡易的CNN模型(不含後面的分類器)

In [ ]:
# 看一下模型摘要
model.summary()

以上，我們透過5個Conv層來做特徵擷取。接著，我們將疊加分類器進入網路，如此才能將擷取到的特徵做分類。

一般來說，分類器會是2至3個Dense層組成的，但是，以下我們以兩個Conv層來取代Dense層：

In [ ]:
conv_layers = [Conv2D(filters=4096, kernel_size=1,
                      strides=1,
                      padding='SAME',
                      activation='relu'),
               Conv2D(filters=4096, kernel_size=1,
                      strides=1,
                      padding='SAME',
                      activation='relu')
              ]

# 將兩層個用來分類的Conv層添加至模型裡。這兩個Conv層取代了一般Dense層的地位。
for layer in conv_layers:
    model.add(layer)

In [ ]:
model.summary()

接著，我們想要再添加一層Conv layer, 將輸出Tensor的形狀變成 ```(None,1,1,2)```。末端維度長之所以改成2，是因為我們想要預測出有/無鯨魚，這兩種可能性。

In [ ]:
# 練習：試著添加Conv2D, 將輸出Tensor形狀改變成 (#samples,1,1,2) 

# model.add( Conv2D(...)
#          )

In [ ]:
model.summary()

---

於訓練階段，圖的大小我們會固定為227 X 227。於此階段，機器要學習出，什麼樣的圖特徵看起來是有鯨魚，什麼樣的圖特徵看起來是沒鯨魚。

之後，於預測階段，我們會丟一張大圖(也許大小是1000 X 2000左右)進來此網路。此網路將能夠預測，此大圖的各局部區域是否存在鯨魚。

因此，我們需要讓模型可以接受任何長寬的圖片。為了達到這個目的，我們必須重建模型，告知其輸入圖片的長寬應為任意大小。

換句話說，我們得將input_shape更改為```(None,None,3)```，然後重建模型。

In [ ]:
# 建立模型
input_shape=(None,None,3)
model=my_ConvNet(input_shape)

# 添加兩層末端分類器
conv_layers = [Conv2D(filters=4096, kernel_size=1,
                      strides=1,
                      padding='SAME',
                      activation='relu'),
               Conv2D(filters=4096, kernel_size=1,
                      strides=1,
                      padding='SAME',
                      activation='relu')
              ]
for layer in conv_layers:
    model.add(layer)

# 添加一層特製的Conv層，它能夠使得最後的Tensor只能紀錄兩個信號：有或無鯨魚。
model.add( Conv2D(2,kernel_size=3))
# 添加Softmax函數，輸出有/無鯨魚的機率。
model.add( SoftmaxMap() )

# 編譯模型，告知模型學習方式
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.01,momentum=0.9,nesterov=True),
              metrics=['accuracy'])

### <a id='32'> II. 訓練模型三步驟：1. 將圖像準備好成為適合訓練的格式 2. 訓練模型 3. 畫出訓練過程 </a>

In [ ]:
%%time

epochs=3
batch_size=128

# 將圖像準備好成為適合訓練的格式 

train_datagen = ImageDataGenerator(
    samplewise_center=True,
    rescale=1./255,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
        samplewise_center=True,
        rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '../datasets/whale2/data/train',
        target_size=(227, 227),
        batch_size=batch_size,
        class_mode='categorical', 
        shuffle=True)

val_generator = val_datagen.flow_from_directory(
        '../datasets/whale2/data/val',
        target_size=(227, 227),
        batch_size=batch_size,
        class_mode='categorical', 
        shuffle=True)

def generatorReshapingLabel(generator):
    '''以flow_from_directory產生的generator其標籤格式不符合我們所需，故我們需重新整理標籤的格式。'''
    num_classes=len(generator.class_indices)
    for x, y in generator:
        yield (x, y.reshape((-1,1,1,num_classes)) )
        
train_generator = generatorReshapingLabel(train_generator)
val_generator   = generatorReshapingLabel(val_generator)

# 訓練模型
history=model.fit_generator(
        train_generator,
        steps_per_epoch=7268 // batch_size,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=1818// batch_size)

# 畫出訓練過程
plt.plot(history.history['acc'],ms=5,marker='o',label='accuracy')
plt.plot(history.history['val_acc'],ms=5,marker='o',label='val accuracy')
plt.legend()
plt.show()

# 儲存模型架構
with open('../checkpoints/first_try_fcn.json', 'w') as jsOut:
    json.dump(model.to_json(), jsOut)
# 儲存訓練好的模型權重
model.save_weights('../checkpoints/first_try_fcn.h5')

### <a id='33'> III. 檢驗模型好壞</a>

In [ ]:
testGen = ImageDataGenerator(
        samplewise_center=True,
        rescale=1./255).flow_from_directory(
        '../datasets/whale2/data/val',
        target_size=(227, 227),
        batch_size=batch_size,
        class_mode='categorical', 
        shuffle=False)

print('\n')
print( testGen.class_indices ,'\n')
yTrue=testGen.classes

predictions_last_epoch=model.predict_generator(generator=testGen,steps=1818// batch_size +1)
yPredicted=predictions_last_epoch.argmax(axis=-1)
yPredicted=yPredicted.reshape(yPredicted.shape[0])
print(classification_report(yTrue , yPredicted, digits = 6))

print('accuracy=',np.sum(yTrue==yPredicted)/len(yTrue))

## <a id='34'>IV. 拿訓練好的 Fully Convolutional Network 來做預測</a>

In [ ]:
def modelLoad(modelPath,weightsPath):
    '''此方法用來載入已經儲存好的模型和模型所學好的權重。'''

    # load the previous stored model
    with open(modelPath,'r') as jsIn:
        modelJson=json.load(jsIn)

    model=model_from_json(modelJson)
    model.load_weights(weightsPath)

    return model

In [ ]:
def inferImagePreprocessing(imagePath):
    '''此方法將一張輸入的大圖轉變成模型能夠拿來使用的格式。'''
    
    # load the image and display it
    image=load_img(imagePath)
    image
    plt.imshow(image)
    plt.show()
    # convert it to the numerical array
    imageArray=img_to_array(image)
    print('shape of the image=',imageArray.shape)

    # reshape the image to 4D, as the shape of our input
    # should be (#samples, height, width, #channels)
    imageArray=imageArray.reshape((-1,*imageArray.shape) )
    print('\nshape of the image=',imageArray.shape)

    # rescale the image
    imgGen = ImageDataGenerator( samplewise_center=True,
                                    rescale=1./255. )
    generator = imgGen.flow( imageArray,
                             batch_size=1,
                             shuffle=False )
    # pick the rescaled image up from the generator
    for idx,figArray in enumerate(generator):
        if idx>0:
            break    
    return figArray

In [ ]:
# 預處理要預測的圖像
figArray=inferImagePreprocessing('../datasets/whale2/data/samples/w_3.jpg')
# 載入先前訓練好的模型之權重
#model.load_weights('../checkpoints/first_try_fcn.h5')
# 用模型來做預測
start = time.time()
prediction = model.predict(figArray)
prediction=prediction[0].argmax(-1)
end = time.time()
# 畫出預測結果
plt.imshow( prediction )
plt.show()
print ('Inference time = %s ' % str(end-start) + ' seconds.')

[回索引](#本筆記內容如下)

-----

## 後記
我們簡介了何謂FCN，以及如何建立，訓練出一個簡單的FCN模型。若想深入了解，或者是嘗試建立出能夠實際拿來應用的模型，我建議你參考以下幾個教學範例(MXNet Gluon)：
* https://gluon-cv.mxnet.io/build/examples_segmentation/demo_fcn.html
* https://gluon-cv.mxnet.io/build/examples_segmentation/train_fcn.html